In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from sklearn.neighbors import NearestNeighbors
from termcolor import colored
from collections import Counter
import json

pd.set_option('display.max_colwidth', None)

In [ ]:
inspections = pd.read_csv("prohlidky.csv")
inspections = inspections.drop(['Unnamed: 0'], axis=1)
inspections.DatKont = pd.to_datetime(inspections.DatKont)
inspections.DatPrvReg = pd.to_datetime(inspections.DatPrvReg)

In [ ]:
#failed attempt to use kNN

# tmp_df_copy = tmp_df[['index', 'timestamp_int']]
# sample = tmp_df_copy.sample()
# neighbours = neigh.kneighbors(sample, 5, return_distance=False)

# for neighbour in neighbours:
#     for neighb in neighbour:
#         display(tmp_df.iloc[[neighb]].DatKont)
        
# tmp_df.where(tmp_df.DatKont.dt.month == 7).dropna(how='all').where(tmp_df.DatKont.dt.day == 23).dropna(how='all').where(tmp_df.DatKont.dt.hour == 11).dropna(how='all')

In [ ]:
class VehicleModel:
    def __init__(self, name, count, neighbours):
        self.name = name
        self.count = count
        self.neighbours = neighbours
        self.neighbours_count = 0

In [ ]:
tmp_df = inspections.where(inspections.STK == 3100).dropna(how='all').sort_values(by=['DatKont']).reset_index()

In [ ]:
display(len(tmp_df.drop_duplicates(['TZn', 'ObchOznTyp']).index))
display(len(tmp_df.drop_duplicates(['TZn']).index))

In [ ]:
vehicles = {}

for TZn in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
    neighbours = {}
    
    for TZnN in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
        neighbours.update({TZnN: 0})
    
    model = VehicleModel(TZn, len(tmp_df[tmp_df.TZn == TZn].index), neighbours)
    vehicles.update({TZn: model})

In [ ]:
#27 min - average inspection duration 

for index, row in tmp_df.iterrows():
    for i, neighbour in tmp_df[(tmp_df.DatKont < (row.DatKont + timedelta(minutes=27))) & (tmp_df.DatKont > (row.DatKont - timedelta(minutes=27)))].iterrows():
        if neighbour.TZn != row.TZn:
            vehicles[row.TZn].neighbours[neighbour.TZn] += 1
            vehicles[row.TZn].neighbours_count +=1

In [ ]:
#ideally should not print anything - just cheking if it went well or not

for vehicle1 in vehicles.keys():
    for vehicle2 in vehicles.keys():
        if vehicles[vehicle1].neighbours[vehicle2] != vehicles[vehicle2].neighbours[vehicle1]:
            print('an error occured')

In [162]:
total = len(tmp_df.index)

suspicious = []

for vehicle1 in vehicles.keys():
    for vehicle2 in vehicles.keys():
        if vehicle1 != vehicle2 and vehicles[vehicle1].neighbours[vehicle2] > 1:     
            
            naive_probability = vehicles[vehicle2].count/total
            estimated = naive_probability*vehicles[vehicle1].neighbours_count
            actual = vehicles[vehicle1].neighbours[vehicle2]
            difference = round(100*abs(naive_probability*vehicles[vehicle1].neighbours_count-vehicles[vehicle1].neighbours[vehicle2])/((naive_probability*vehicles[vehicle1].neighbours_count+vehicles[vehicle1].neighbours[vehicle2])/2))
            
            if difference > 30 and actual > estimated and actual > 10:
                print(vehicle1, ' and ', vehicle2, ':')
                print('Estimated conjunctions count: ', estimated)
                print('Actual conjunction count: ', actual)
                print(colored('Difference: {0}%'.format(difference), 'red' if difference > 60 else 'yellow'))
                print('----------------------------')
                
                suspicious.append(vehicle1)

FORD  and  AVIA :
Estimated conjunctions count:  38.07799293891088
Actual conjunction count:  52
Difference: 31%
----------------------------
FORD  and  TATRA :
Estimated conjunctions count:  20.67091045255162
Actual conjunction count:  29
Difference: 34%
----------------------------
FORD  and  DAEWOO :
Estimated conjunctions count:  15.23119717556435
Actual conjunction count:  21
Difference: 32%
----------------------------
FORD  and  VLASTNI VYROBA :
Estimated conjunctions count:  17.951053814057985
Actual conjunction count:  25
Difference: 33%
----------------------------
FORD  and  SPORT  JACHT :
Estimated conjunctions count:  20.126939124852893
Actual conjunction count:  30
Difference: 39%
----------------------------
FORD  and  ISUZU :
Estimated conjunctions count:  6.5276559323847225
Actual conjunction count:  11
Difference: 51%
----------------------------
FORD  and  MERCEDES :
Estimated conjunctions count:  11.423397881673264
Actual conjunction count:  16
Difference: 33%
-----

PORSCHE  and  ITALMOTO :
Estimated conjunctions count:  0.34321172568738634
Actual conjunction count:  11
Difference: 188%
----------------------------
SETRA  and  AUDI :
Estimated conjunctions count:  18.705199529260724
Actual conjunction count:  26
Difference: 33%
----------------------------
SETRA  and  SUZUKI :
Estimated conjunctions count:  7.48207981170429
Actual conjunction count:  13
Difference: 54%
----------------------------
YAMAHA  and  HYUNDAI :
Estimated conjunctions count:  13.993527334973788
Actual conjunction count:  29
Difference: 70%
----------------------------
YAMAHA  and  FIAT :
Estimated conjunctions count:  9.762223173210655
Actual conjunction count:  15
Difference: 42%
----------------------------
BOMBARDIER  and  SKODA :
Estimated conjunctions count:  19.25510859099176
Actual conjunction count:  27
Difference: 33%
----------------------------
DAEWOO  and  RENAULT :
Estimated conjunctions count:  7.340483577618487
Actual conjunction count:  13
Difference: 56%
-

In [163]:
values, counts = np.unique(suspicious, return_counts=True)
suspicious_sorted = dict(zip(values, counts))
suspicious_sorted = {k: v for k, v in sorted(suspicious_sorted.items(), key=lambda item: item[1], reverse=True)}

In [164]:
suspicious_sorted

{'SKODA': 40,
 'MERCEDES-BENZ': 17,
 'FORD': 12,
 'RENAULT': 7,
 'VW': 6,
 'AUDI': 5,
 'PEUGEOT': 5,
 'VOLVO': 5,
 'IVECO': 4,
 'JEEP': 4,
 'OPEL': 4,
 'BMW': 3,
 'FIAT': 3,
 'HYUNDAI': 3,
 'KRONE': 3,
 'MAN': 3,
 'MAZDA': 3,
 'SCANIA': 3,
 'SCHMITZ': 3,
 'AGADOS': 2,
 'ALFA ROMEO': 2,
 'CHEVROLET': 2,
 'ITALMOTO': 2,
 'KIA': 2,
 'LAND ROVER': 2,
 'MERCEDES': 2,
 'PANAV': 2,
 'SETRA': 2,
 'SUBARU': 2,
 'TOYOTA': 2,
 'VLASTNI VYROBA': 2,
 'VOLKSWAGEN': 2,
 'YAMAHA': 2,
 'APRILIA': 1,
 'BOMBARDIER': 1,
 'CITROEN': 1,
 'D TEC': 1,
 'DAEWOO': 1,
 'DAF': 1,
 'HARLEY-DAVIDSON': 1,
 'HUMBAUR': 1,
 'IRIZAR': 1,
 'ISUZU': 1,
 'JAGUAR': 1,
 'JAWA': 1,
 'KOGEL': 1,
 'LADA': 1,
 'LANCIA': 1,
 'LOHR': 1,
 'MERCEDES-AMG': 1,
 'MINI': 1,
 'MULTICAR': 1,
 'NISSAN': 1,
 'OTOKAR': 1,
 'PORSCHE': 1,
 'ROVER': 1,
 'SCHWARZMULLER': 1,
 'SIMSON': 1,
 'SMART': 1,
 'SPORT  JACHT': 1,
 'SUZUKI': 1,
 'SVAN': 1,
 'TRABANT': 1,
 'TRIUMPH': 1,
 'VAZ': 1}

In [166]:
#final script for all of the stations

class StationConjunctionsInfo:
    def __init__(self, stk_id, suspicious_sorted, vehicles, inspection_count):
        self.stk_id = stk_id
        self.suspicious_sorted = suspicious_sorted
        self.vehicles = vehicles
        self.inspection_count = inspection_count

station_conjunctions_infos = {}

for stk_id in inspections.STK.unique():
    tmp_df = inspections.where(inspections.STK == stk_id).dropna(how='all').sort_values(by=['DatKont']).reset_index()
    
    vehicles = {}

    for TZn in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
        neighbours = {}

        for TZnN in tmp_df.drop_duplicates(['TZn']).TZn.to_numpy():
            neighbours.update({TZnN: 0})

        model = VehicleModel(TZn, len(tmp_df[tmp_df.TZn == TZn].index), neighbours)
        vehicles.update({TZn: model})
        
    #27 min - average inspection duration 

    for index, row in tmp_df.iterrows():
        for i, neighbour in tmp_df[(tmp_df.DatKont < (row.DatKont + timedelta(minutes=27))) & (tmp_df.DatKont > (row.DatKont - timedelta(minutes=27)))].iterrows():
            if neighbour.TZn != row.TZn:
                vehicles[row.TZn].neighbours[neighbour.TZn] += 1
                vehicles[row.TZn].neighbours_count +=1
                
    total = len(tmp_df.index)

    suspicious = []

    for vehicle1 in vehicles.keys():
        for vehicle2 in vehicles.keys():
            if vehicle1 != vehicle2 and vehicles[vehicle1].neighbours[vehicle2] > 1:     

                naive_probability = vehicles[vehicle2].count/total
                estimated = naive_probability*vehicles[vehicle1].neighbours_count
                actual = vehicles[vehicle1].neighbours[vehicle2]
                difference = round(100*abs(naive_probability*vehicles[vehicle1].neighbours_count-vehicles[vehicle1].neighbours[vehicle2])/((naive_probability*vehicles[vehicle1].neighbours_count+vehicles[vehicle1].neighbours[vehicle2])/2))

                if difference > 30 and actual > estimated and actual > 10:
                    suspicious.append(vehicle1)
                    
    values, counts = np.unique(suspicious, return_counts=True)
    suspicious_sorted = dict(zip(values, counts))
    suspicious_sorted = {k: v for k, v in sorted(suspicious_sorted.items(), key=lambda item: item[1], reverse=True)}
    
    result = StationConjunctionsInfo(stk_id, suspicious_sorted, vehicles, total)
    station_conjunctions_infos.update({stk_id : result})
    print('Station', stk_id, 'finished.')

Station 3122 finished.
Station 3205 finished.
Station 3114 finished.
Station 3618 finished.
Station 3748 finished.
Station 3846 finished.
Station 3307 finished.
Station 3755 finished.
Station 3124 finished.
Station 3710 finished.
Station 3213 finished.
Station 3706 finished.
Station 3722 finished.
Station 3828 finished.
Station 3402 finished.
Station 3320 finished.
Station 3765 finished.
Station 3106 finished.
Station 3648 finished.
Station 3815 finished.
Station 3102 finished.
Station 3805 finished.
Station 3804 finished.
Station 3768 finished.
Station 3719 finished.
Station 3712 finished.
Station 3531 finished.
Station 3127 finished.
Station 3516 finished.
Station 3641 finished.
Station 3754 finished.
Station 3417 finished.
Station 3851 finished.
Station 3829 finished.
Station 3243 finished.
Station 3100 finished.
Station 3314 finished.
Station 3507 finished.
Station 3732 finished.
Station 3752 finished.
Station 3303 finished.
Station 3413 finished.
Station 3520 finished.
Station 321

Station 3751 finished.
Station 8409 finished.
Station 6721 finished.
Station 6811 finished.
Station 6305 finished.
Station 6701 finished.
Station 6505 finished.
Station 8730 finished.
Station 9999 finished.
Station 8402 finished.
Station 3533 finished.
Station 8270 finished.
Station 6815 finished.
Station 6813 finished.
Station 8811 finished.
Station 8611 finished.
Station 8601 finished.
Station 8705 finished.
Station 3246 finished.
Station 6616 finished.
Station 6738 finished.
Station 8602 finished.
Station 8625 finished.
Station 8451 finished.
Station 8861 finished.
Station 8748 finished.
Station 6409 finished.
Station 8551 finished.
Station 8808 finished.
Station 8605 finished.
Station 8630 finished.
Station 8742 finished.
Station 8404 finished.
Station 8713 finished.
Station 8335 finished.
Station 8708 finished.
Station 8408 finished.
Station 8855 finished.
Station 8665 finished.
Station 8541 finished.
Station 8661 finished.
Station 8307 finished.
Station 8204 finished.
Station 855

In [187]:
str(station_conjunctions_infos[3848].suspicious_sorted)

"{'SKODA': 30, 'FIAT': 7, 'FORD': 6, 'PEUGEOT': 5, 'HONDA': 4, 'SUZUKI': 3, 'TOYOTA': 3, 'AUDI': 2, 'JAWA': 2, 'KIA': 2, 'NISSAN': 2, 'OPEL': 2, 'VOLKSWAGEN': 2, 'VOLVO': 2, 'AGADOS': 1, 'ALFA ROMEO': 1, 'BMW': 1, 'CITROEN': 1, 'DACIA': 1, 'DAEWOO': 1, 'HYUNDAI': 1, 'KAWASAKI': 1, 'MAZDA': 1, 'MINI': 1, 'PIAGGIO': 1, 'RENAULT': 1, 'SPORT  JACHT': 1, 'SUBARU': 1, 'VLASTNI VYROBA': 1, 'VW': 1}"

In [190]:
station_conjunctions_infos[3848].vehicles

{'SKODA': <__main__.VehicleModel at 0x1a0da0cfb48>,
 'PEUGEOT': <__main__.VehicleModel at 0x1a0ecf46148>,
 'HYUNDAI': <__main__.VehicleModel at 0x1a0e2b5c2c8>,
 'RENAULT': <__main__.VehicleModel at 0x1a094616b48>,
 'SEAT': <__main__.VehicleModel at 0x1a0da0b0b48>,
 'NISSAN': <__main__.VehicleModel at 0x1a0ecf46e48>,
 'VW': <__main__.VehicleModel at 0x1a0d7a58ec8>,
 'FORD': <__main__.VehicleModel at 0x1a0de514148>,
 'HONDA': <__main__.VehicleModel at 0x1a0ecf45b88>,
 'CHEVROLET': <__main__.VehicleModel at 0x1a094618f08>,
 'ALFA ROMEO': <__main__.VehicleModel at 0x1a0ef927748>,
 'BMW': <__main__.VehicleModel at 0x1a0ef93a608>,
 'TOYOTA': <__main__.VehicleModel at 0x1a0bf0e0d48>,
 'PONGRATZ': <__main__.VehicleModel at 0x1a0fd129208>,
 'VOLKSWAGEN': <__main__.VehicleModel at 0x1a0946180c8>,
 'DACIA': <__main__.VehicleModel at 0x1a0da0cd288>,
 'OPEL': <__main__.VehicleModel at 0x1a0ef938888>,
 'CITROEN': <__main__.VehicleModel at 0x1a0ef938048>,
 'LAND ROVER': <__main__.VehicleModel at 0x1a

In [178]:
len(station_conjunctions_infos)

552

In [ ]:
stk_ids = []
insp_count = []
suspicious_sorted = []

for stk_info in station_conjunctions_infos.keys():
    